# GAによる関数最大値探索 

### 1.必要モジュールのインポート

In [1]:
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

### 2.GAのシミュレーションクラスと個体（染色体）クラスの生成

In [2]:
class SimpleGA:
    def __init__(self):
        self.N = 10 # 個体数
        self.ITERATION = 100 # 世代数
        self._initialize()

    def _initialize(self):
        self.pool = [Chromosome() for i in range(self.N)]

    def _mutateAll(self):
        for i in range(self.N):
            self.pool[i].mutate()

    def _tournamentSelection(self):
        pool_next = []
        while len(pool_next) < self.N:
            offspring1 = copy.deepcopy(self.pool[random.randrange(self.N)])
            offspring2 = copy.deepcopy(self.pool[random.randrange(self.N)])
            if offspring1.getFittness() > offspring2.getFittness():
                pool_next.append(offspring1)
            else:
                pool_next.append(offspring2)
        self.pool = pool_next[:]

    def _printStatus(self, iteration):
        print("generation\t" + str(iteration))
        for c in self.pool:
            print("\t" + str(c))

    def evolve(self):
        for i in range(self.ITERATION):
            self._printStatus(i)
            self._tournamentSelection()
            self._mutateAll()

In [3]:
class Chromosome:
    def __init__(self):
        self.LENGTH = 8 # 遺伝子数
        self.MUTATION_RATE = 0.05 # 突然変異率
        self.gene = [random.randint(0,1) for i in range(self.LENGTH)]

    def mutate(self):
        for i in range(self.LENGTH):
            if random.random() < self.MUTATION_RATE:
                self.gene[i] = 1 - self.gene[i]
    
    def getVal(self): # 遺伝子型から表現型への発現
        value = 0.
        for g in self.gene:
            value *= 2
            value += g
        value = value / (2 ** self.LENGTH - 1.0) *4
        return value
    
    def getFittness(self): # 適応度評価
        result = self.getVal()
        return  -1.5*(pow(result-1/2,4) -16/3*pow(result-1/2,3)+6*pow(result-1/2,2)+5)

    def __str__(self):
        result = ""
        for g in self.gene:
            result += str(g)
        result += "\t" + str(self.getFittness())
        return result

In [4]:
ga = SimpleGA()
ga.evolve()

generation	0
	11101010	5.4252817596771905
	00001110	-8.393204371333535
	00011101	-7.51904447807981
	10001000	-7.3265944444444475
	10111000	1.3189825854176291
	11000101	3.6230096852776192
	11011100	5.957916646711581
	00111100	-8.621602276672931
	11101111	4.723619261844519
	00001010	-8.903700325200981
	00101110	-7.858429902991795
	10000100	-7.834035154631769
	00001101	-8.508128648084515
	11100100	5.890217380658747
	10101001	-1.6343959208529117
	10000100	-7.834035154631769
	01101111	-9.628021816668863
	00110001	-8.002182343163195
	01011000	-9.917893005418463
	01111100	-8.693840355984989
generation	1
	10101001	-1.6343959208529117
	10111000	1.3189825854176291
	11100100	5.890217380658747
	10001001	-7.19202042424903
	11000101	3.6230096852776192
	10011100	-4.126304069036527
	10011101	-3.9426911014395536
	11101010	5.4252817596771905
	10000100	-7.834035154631769
	11011100	5.957916646711581
	00101110	-7.858429902991795
	11101111	4.723619261844519
	11011100	5.957916646711581
	10101001	-1.634395920

## SimpleGAクラスを継承したアニメーション用クラスの生成

In [5]:
%matplotlib nbagg
class SimpleGA_animation(SimpleGA):
    # コンストラクタ
    def __init__(self):
        super(SimpleGA_animation, self).__init__()
        self.fig = plt.figure(figsize=(5, 5))
        self.ims = []
    
    # SimpleGAのevolveメソッドをオーバーライド
    def evolve(self):
        for i in range(self.ITERATION):
            # self._printStatus(i)
            # self._printAverage()
            self._tournamentSelection()
            self._mutateAll()
            self.snapshot(i)
        self.output_animation()
    
    def maxscore(self):
        maxs = self.pool[0].getFittness()
        for i in range(self.N):
            if self.pool[i].getFittness() > maxs:
                maxs = self.pool[i].getFittness()
        return maxs
    
    # スナップショット
    def snapshot(self, t):
        val_list=[]
        im=plt.plot(np.linspace(0,4,500), [-1.5*(pow(n-1/2,4) -16/3*pow(n-1/2,3)+6*pow(n-1/2,2)+5) for n in np.linspace(0,4,500)], color='black', linestyle='solid')
        plt.title("GA:遺伝的アルゴリズムによるシミュレーション", fontname="MS Gothic")
        im += [plt.text(0,2,'N = {}'.format(self.N))]
        im += [plt.text(0,3,'generation: {}'.format(t))]
        im += [plt.text(0,4,'maxscore: {}'.format(self.maxscore()))]
        im += [plt.scatter([chromosome.getVal() for chromosome in self.pool], [chromosome.getFittness() for chromosome in self.pool],c='b')]
        
        self.ims.append(im)
        
    # 出力
    def output_animation(self):
        global ani # Jupyterでアニメーション表示のためにglobalが必要
        ani = animation.ArtistAnimation(self.fig, self.ims)
        # ani.save('./anim.mp4', writer="ffmpeg")
        ani.save('./anim3.gif', writer="pillow")

### アニメーション

In [6]:
ga = SimpleGA_animation()
ga.evolve()

<IPython.core.display.Javascript object>